In [1]:
%load_ext autoreload
%autoreload

## This expands a notebook to full width
from IPython.display import display, HTML

display(HTML("<style>.container { width:90% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))
## Show Python Version
import sys
print("Python: {0}".format(sys.version))

## Show Current Time
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.10.9 | packaged by conda-forge | (main, Jan 11 2023, 15:15:40) [MSC v.1916 64 bit (AMD64)]
Notebook Last Run Initiated: 2023-03-16 21:50:39.675346


In [6]:
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabasz_score
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns

import pickle

import matplotlib.pyplot as plt

In [7]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go

np.random.seed(1)

In [8]:
# Input vector, returns nearest customer(s)
def Cosine_Similarity(centroid,weight,word_to_index,vocab_size,index_to_word,threshold):
              
    customer_similarity_index = []
    customer_similarity_theta = []

    for i in range(vocab_size):
        
        customer = weight.iloc[i,:]
        
        theta_sum = np.dot(centroid, customer)
        theta_den = np.linalg.norm(centroid) * np.linalg.norm(customer)
        theta = theta_sum / theta_den
        
        if theta >= threshold:
            customer_similarity_index.append(i)
            customer_similarity_theta.append(theta)
        
    return customer_similarity_index, customer_similarity_theta

In [9]:
VOCAB_SIZE = 12171

In [10]:
data      = pd.read_csv('data/data4analytics.csv')
centroids = pd.read_csv('data/kmeans_centroids.csv')

In [11]:
with open('data/word_to_index_v3.pkl', 'rb') as fp:
    word_to_index = pickle.load(fp)
    
with open('data/index_to_word_v3.pkl', 'rb') as fp:
    index_to_word = pickle.load(fp)    

In [12]:
data.head()

,accountid,userid,col0,col1,col2,col3,col4,col5,col6,col7,...,age,usercreationdate,engaged,camera_count,firsttransaction,ratio_open30,cluster_x,target,counter,cluster_y
0,B24CB6E7-C8E1-4B8D-9778-B724F67E05B2,64841C33-4A5A-4E8C-BB61-BDD70F326E90,-0.223028,0.168989,0.049634,0.265127,-0.099026,0.110539,0.082398,-0.217806,...,age_unk,new_user,strongly_engaged,single_camera,fast,low_frequency_ratio30,0,1,1,1
1,F51FC1D9-5D37-4D34-AB54-1F41B9BD8C06,F51FC1D9-5D37-4D34-AB54-1F41B9BD8C06,0.089386,-0.060354,0.026696,-0.238734,-0.059884,0.076348,0.078483,0.028509,...,age_adult,established_user,strongly_engaged,multi_camera,slow,mediumfrequencyratio30,0,1,1,0
2,2036E661-F89B-46CA-AD81-1AD78CA8C91E,913AEF55-040B-4F53-BA0A-7F136B4368F6,-0.133327,-0.076570,-0.103270,-0.255172,-0.033821,0.102381,0.228990,0.050512,...,age_adult,new_user,strongly_engaged,single_camera,immediate,low_frequency_ratio30,0,1,1,0
3,565A44C3-7F0A-4F50-89AA-7E296917B8FF,7584C122-1E64-47E8-B587-F466A39FD7D4,-0.137795,0.058056,-0.039041,0.080276,-0.143586,0.257206,0.243565,-0.017286,...,age_adult,new_user,strongly_engaged,single_camera,immediate,mediumfrequencyratio30,0,1,1,0
4,677AEC7D-BC45-4D29-B116-3758EA2D15CB,4C70B81B-2A69-4708-A7CE-C1BBCC03E043,-0.038019,-0.021056,0.143451,0.090723,0.119178,0.086459,0.210279,-0.043040,...,age_unk,new_user,strongly_engaged,single_camera,immediate,mediumfrequencyratio30,0,1,1,1


In [13]:
cluster0 = centroids.iloc[0,:]
cluster1 = centroids.iloc[1,:]

In [14]:
weights = data.iloc[:,2:302]

In [47]:
index, thetas = Cosine_Similarity(cluster0,weights,word_to_index,VOCAB_SIZE,index_to_word,0.915)

In [48]:
Similarity_cluster0 = []
for i in index:
    Similarity_cluster0.append(data.iloc[i,302:-4])

In [49]:
Similarity_cluster0 = pd.DataFrame(Similarity_cluster0)
Similarity_cluster0['theta'] = thetas

In [50]:
Similarity_cluster0.sort_values(by=['theta'],ascending=False)

,postal_code,city,county,state,rural_code,key_zips_ind,gender,marital_status,age,usercreationdate,engaged,camera_count,firsttransaction,ratio_open30,theta
7156,98208,everett,snohomish,wa,rc_u,keyservice,gender_other,ms_other,age_unk,new_user,strongly_engaged,single_camera,immediate,mediumfrequencyratio30,0.928749
9242,75034,frisco,denton,tx,rc_u,keyservice,gender_m,ms_m,age_senior,new_user,strongly_engaged,single_camera,immediate,mediumfrequencyratio30,0.922482
5889,98270,marysville,snohomish,wa,rc_u,keyservice,gender_m,ms_m,age_adult,new_user,strongly_engaged,single_camera,immediate,mediumfrequencyratio30,0.921998
11400,75065,lake_dallas,denton,tx,rc_s,keyservice,gender_other,ms_other,age_unk,new_user,strongly_engaged,single_camera,immediate,mediumfrequencyratio30,0.920925
4763,98203,everett,snohomish,wa,rc_u,keyservice,gender_other,ms_other,age_unk,new_user,strongly_engaged,single_camera,immediate,mediumfrequencyratio30,0.919733
5955,76266,sanger,denton,tx,rc_s,keyservice,gender_other,ms_other,age_unk,new_user,strongly_engaged,single_camera,immediate,mediumfrequencyratio30,0.918635
5734,22046,falls_church,falls_church_city,va,rc_u,keyservice,gender_other,ms_other,age_unk,new_user,strongly_engaged,single_camera,immediate,mediumfrequencyratio30,0.915748
2174,92069,san_marcos,san_diego,ca,rc_u,keyservice,gender_m,ms_m,age_senior,new_user,strongly_engaged,single_camera,immediate,mediumfrequencyratio30,0.915325
3321,30019,dacula,gwinnett,ga,rc_s,keyservice,gender_other,ms_other,age_unk,new_user,strongly_engaged,single_camera,immediate,mediumfrequencyratio30,0.915317
10997,92880,corona,riverside,ca,rc_s,keyservice,gender_other,ms_other,age_unk,new_user,strongly_engaged,single_camera,immediate,mediumfrequencyratio30,0.915274


In [43]:
index, thetas = Cosine_Similarity(cluster1,weights,word_to_index,VOCAB_SIZE,index_to_word,0.925)

In [44]:
Similarity_cluster1 = []
for i in index:
    Similarity_cluster1.append(data.iloc[i,302:-4])

In [45]:
Similarity_cluster1 = pd.DataFrame(Similarity_cluster1)
Similarity_cluster1['theta'] = thetas

In [46]:
Similarity_cluster1.sort_values(by=['theta'],ascending=False)

,postal_code,city,county,state,rural_code,key_zips_ind,gender,marital_status,age,usercreationdate,engaged,camera_count,firsttransaction,ratio_open30,theta
5844,77471,rosenberg,fort_bend,tx,rc_s,keyservice,gender_other,ms_other,age_unk,new_user,engaged,single_camera,slow,mediumfrequencyratio30,0.934205
6893,55109,saint_paul,ramsey,mn,rc_u,keyservice,gender_other,ms_other,age_unk,new_user,engaged,single_camera,slow,mediumfrequencyratio30,0.931681
2560,78753,austin,travis,tx,rc_u,keyservice,gender_m,ms_m,age_senior,new_user,engaged,single_camera,slow,mediumfrequencyratio30,0.929603
9115,33437,boynton_beach,palm_beach,fl,rc_u,keyservice,gender_other,ms_other,age_unk,new_user,engaged,single_camera,slow,mediumfrequencyratio30,0.929432
6831,44505,youngstown,mahoning,oh,rc_s,keyservice,gender_other,ms_other,age_unk,new_user,engaged,single_camera,slow,mediumfrequencyratio30,0.929033
4655,10502,ardsley,westchester,ny,rc_u,keyservice,gender_m,ms_m,age_senior,new_user,engaged,single_camera,slow,mediumfrequencyratio30,0.928180
5689,36526,daphne,baldwin,al,rc_s,keyservice,gender_other,ms_other,age_unk,new_user,engaged,single_camera,slow,mediumfrequencyratio30,0.927570
9745,22042,falls_church,fairfax,va,rc_u,keyservice,gender_other,ms_other,age_unk,new_user,engaged,single_camera,slow,highfrequencyratio30,0.926331
2888,75134,lancaster,dallas,tx,rc_s,keyservice,gender_other,ms_other,age_unk,new_user,engaged,single_camera,slow,mediumfrequencyratio30,0.925998
10309,55426,minneapolis,hennepin,mn,rc_u,keyservice,gender_other,ms_other,age_unk,new_user,engaged,single_camera,slow,low_frequency_ratio30,0.925732


In [60]:
unk = weights.iloc[0,:]

In [69]:
index, thetas = Cosine_Similarity(unk,weights,word_to_index,VOCAB_SIZE,index_to_word,0.915)

In [70]:
Similarity_unk = []
for i in index:
    Similarity_unk.append(data.iloc[i,302:-4])

In [71]:
Similarity_unk = pd.DataFrame(Similarity_unk)
Similarity_unk['theta'] = thetas

In [72]:
Similarity_unk.sort_values(by=['theta'],ascending=False)

,postal_code,city,county,state,rural_code,key_zips_ind,gender,marital_status,age,usercreationdate,engaged,camera_count,firsttransaction,ratio_open30,theta
0,85226,chandler,maricopa,az,rc_s,keyservice,gender_other,ms_other,age_unk,new_user,strongly_engaged,single_camera,fast,low_frequency_ratio30,1.000000
9420,85226,chandler,maricopa,az,rc_s,keyservice,gender_other,ms_other,age_unk,new_user,strongly_engaged,single_camera,fast,low_frequency_ratio30,1.000000
1479,85226,chandler,maricopa,az,rc_s,keyservice,gender_other,ms_other,age_unk,new_user,strongly_engaged,single_camera,immediate,mediumfrequencyratio30,0.945036
3706,85226,chandler,maricopa,az,rc_s,keyservice,gender_other,ms_other,age_unk,new_user,strongly_engaged,single_camera,immediate,mediumfrequencyratio30,0.945036
11962,85028,phoenix,maricopa,az,rc_s,keyservice,gender_other,ms_other,age_unk,new_user,strongly_engaged,single_camera,fast,low_frequency_ratio30,0.940620
5886,85248,chandler,maricopa,az,rc_s,keyservice,gender_other,ms_other,age_unk,established_user,strongly_engaged,single_camera,slow,low_frequency_ratio30,0.917014
2078,85048,phoenix,maricopa,az,rc_s,keyservice,gender_other,ms_other,age_unk,new_user,strongly_engaged,single_camera,immediate,low_frequency_ratio30,0.916225
